In [ ]:
from aoc import *
import re
import os
from aocd.models import Puzzle as AOCDPuzzle


def Puzzle(day, year=2023):
    return AOCDPuzzle(year=year, day=day)

In [ ]:
p = Puzzle(day=1)

In [ ]:
WORDS = {
    "one": 1,
    "two": 2,
    "three": 3,
    "four": 4,
    "five": 5,
    "six": 6,
    "seven": 7,
    "eight": 8,
    "nine": 9,
}


def words_to_digits(s: str):
    while True:
        first = 10**9
        word = None
        digit = None
        for w, d in WORDS.items():
            idx = s.find(w)
            if idx >= 0 and idx < first:
                first, word, digit = idx, w, d
        if word is None:
            break
        s = s.replace(word, str(digit), 1)
    return s


def trebuchet(x):
    def digits(line):
        return [int(d) for d in line if d.isdigit()]

    lines = [digits(line) for line in x.splitlines()]
    return sum(digits[0] * 10 + digits[-1] for digits in lines)


assert trebuchet(p.examples[0].input_data) == 142
assert (
    trebuchet(
        words_to_digits(
            """two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen"""
        )
    )
    == 281
)

In [ ]:
print(trebuchet(p.input_data), trebuchet(words_to_digits(p.input_data)))

In [ ]:
p = Puzzle(day=2)

In [ ]:
def parse_cube_game(line):
    def parse_round(r):
        return dict(
            (tpl[1], int(tpl[0]))
            for tpl in [tuple(x for x in t.split(" ")) for t in r.split(", ")]
        )

    game, rest = line.strip().split(": ")
    game = int(game[5:])
    rounds = [parse_round(r) for r in rest.split("; ")]
    return game, rounds


def valid_round(round: dict, cubes: dict):
    return all(balls <= cubes[color] for color, balls in round.items())


def cube_game(data, cubes: dict = {"red": 12, "green": 13, "blue": 14}):
    valid_games = 0
    total_power = 0
    for line in data.splitlines():
        game, rounds = parse_cube_game(line)
        if all(valid_round(r, cubes) for r in rounds):
            valid_games += game
        min_balls = defaultdict(int)
        for r in rounds:
            for color, count in r.items():
                min_balls[color] = max(min_balls[color], count)
        power = min_balls["red"] * min_balls["green"] * min_balls["blue"]
        total_power += power
    return valid_games, total_power


assert (8, 2286) == cube_game(p.examples[0].input_data)

In [ ]:
p.answers = cube_game(p.input_data)

In [ ]:
p = Puzzle(day=3)

In [ ]:
def parse_gears(input):
    total = 0
    gear_ratio = 0
    schematic = [l.strip() for l in input.splitlines()]
    gears = defaultdict(set)

    def is_part(row, start, end):
        result = False
        for x in range(start, end):
            for xx, yy in neighbors8((x, row)):
                if xx < 0 or xx >= len(line) or yy < 0 or yy >= len(schematic):
                    continue
                n = schematic[yy][xx]
                if n.isdigit() or n == ".":
                    continue
                if n == "*":
                    gears[(xx, yy)].add((row, start, int(schematic[row][start:end])))
                result = True
        return result

    for row, line in enumerate(schematic):
        for m in re.finditer(r"\d+", line):
            start, end = m.span()
            num = int(line[start:end])
            if is_part(row, start, end):
                total += num
    for pos, parts in gears.items():
        if len(parts) != 2:
            continue
        parts = list(parts)
        gear_ratio += parts[0][-1] * parts[1][-1]
    return total, gear_ratio


assert (4361, 467835) == parse_gears(p.examples[0].input_data)

In [ ]:
p.answers = parse_gears(p.input_data)

In [ ]:
p = Puzzle(day=4)

In [ ]:
def scratchcards(data):
    total = 0
    copies = defaultdict(lambda: 1)
    for line in data.splitlines():
        m = re.match(r"^Card\s+(\d+): ([\d\s]+) \| ([\d\s]+)", line.strip())
        assert m, line
        card, winners, have = (
            int(m.group(1)),
            set(vector(m.group(2))),
            set(vector(m.group(3))),
        )
        hits = len(winners & have)
        points = 2 ** (hits - 1) if hits > 0 else 0
        total += points
        copies[card]
        for x in range(card + 1, card + hits + 1):
            copies[x] += copies[card]
    return total, sum(copies.values())


assert (13, 30) == scratchcards(p.examples[0].input_data)

In [ ]:
p.answers = scratchcards(p.input_data)

In [ ]:
p = Puzzle(day=5)

In [ ]:
def parse_seeds(data):
    lines = data.splitlines()
    seeds = vector(lines[0][6:])
    maps = []
    m = None
    for line in lines[2:]:
        if not line.strip():
            continue
        if line.endswith("map:"):
            maps.append([])
            m = maps[-1]
            continue
        m.append(vector(line))
    return seeds, maps


def map_val(val, m):
    for dest, src, length in m:
        offset = val - src
        if 0 <= offset < length:
            return dest + offset
    return val


def map_range(start: int, count: int, m: tuple[int]):
    queue = [(start, count)]
    while queue:
        (start, count) = queue.pop()
        for dest, src, length in m:

            def translate(s):
                offset = s - src
                assert offset >= 0, f"translate: {s} on {(dest, src, length)}"
                return dest + offset

            x = max(src, start)
            y = min(src + length, start + count)
            if not x < y:
                continue
            yield translate(x), y - x
            if x > start:
                queue.append((start, x - start))
            if y < start + count:
                queue.append((y, start + count - y))
            break
        else:
            yield (start, count)


def start_garden(seeds, maps):
    best = BIG
    for dest in seeds:
        for m in maps:
            dest = map_val(dest, m)
        best = min(best, dest)
    part2 = BIG
    for start, count in grouper(seeds, 2):
        if count > 1_000_000:
            continue
        for dest in range(start, start + count):
            for m in maps:
                dest = map_val(dest, m)
            part2 = min(part2, dest)
    return best, part2


assert (35, 46) == start_garden(*parse_seeds(p.examples[0].input_data))

In [ ]:
p.answer_a = start_garden(*parse_seeds(p.input_data))[0]

In [ ]:
seeds, maps = parse_seeds(p.input_data)
best = 10**10
for node, count in grouper(seeds, 2):
    dests = [(node, count)]
    for i, m in enumerate(maps, 1):
        dests = [x for d in dests for x in map_range(*d, m)]
    best = min(best, min(dests)[0])

p.answer_b = best

In [ ]:
p = Puzzle(day=6)

In [ ]:
def boat_race(data):
    d = array(data)
    races = dict(zip(d[0][1:], d[1][1:]))

    def ways_to_win(time, distance):
        wins = 0
        winning = False
        for speed in range(time + 1):
            if speed * (time - speed) > distance:
                wins += 1
                winning = True
                continue
            if winning:
                break
        return wins

    results = []
    for time, distance in races.items():
        results.append(ways_to_win(time, distance))
    part_a = multiply(results)
    d = array(data.replace(" ", ""))
    races = dict(zip(d[0][1:], d[1][1:]))
    results = []
    for time, distance in races.items():
        results.append(ways_to_win(time, distance))
    return part_a, multiply(results)


assert (288, 71503) == boat_race(p.examples[0].input_data)

In [ ]:
p.answers = boat_race(p.input_data)

In [ ]:
p = Puzzle(day=7)

In [ ]:
def card_rank(card: str, jokers: bool = False):
    if not jokers:
        return "AKQJT98765432".index(card)
    return "AKQT98765432J".index(card)


def hand_order(hand: str, jokers: bool = False):
    return tuple(card_rank(c, jokers) for c in hand)


def hand_type(hand: str, use_jokers: bool = False):
    c = Counter(hand)
    if use_jokers:
        # Eliminate Jokers, add them to card with highest count
        num_jokers = c.get("J", 0)
        if num_jokers < 5:  # We have at least one other card
            c.subtract({"J": num_jokers})
            card = c.most_common(1)[0][0]
            c.update({card: num_jokers})
    counts = c.most_common()
    best = counts[0][1]
    if best == 5 or best == 4 or (best == 3 and counts[1][1] == 2):
        return 5 - best
    if best == 3:  # Not a full house by the test above
        return 3
    if best == 2 and counts[1][1] == 2:
        return 4
    if best == 2:
        return 5
    return 6


def camel_cards(data):
    hands = array(data)
    ordered = sorted(hands, key=lambda t: hand_order(str(t[0])))
    ranked = sorted(ordered, key=lambda t: hand_type(str(t[0])))
    part_a = sum([i * t[1] for i, t in enumerate(reversed(ranked), 1)])

    ordered = sorted(hands, key=lambda t: hand_order(str(t[0]), True))
    ranked = sorted(ordered, key=lambda t: hand_type(str(t[0]), True))
    part_b = sum([i * t[1] for i, t in enumerate(reversed(ranked), 1)])
    return part_a, part_b


assert 6440, 5905 == camel_cards(p.examples[0].input_data)

In [ ]:
p.answers = camel_cards(p.input_data)

In [ ]:
p = Puzzle(day=8)

In [ ]:
def wasteland(data):
    path, _, *rest = data.splitlines()
    graph = dict()
    for line in rest:
        m = re.match(r"(\w+) = \((\w+), (\w+)\)", line)
        assert m
        node, left, right = m.groups()
        graph[node] = (left, right)
    assert "AAA" in graph and "ZZZ" in graph
    node, num_steps = "AAA", 0
    while node != "ZZZ":
        index = 0 if path[num_steps % len(path)] == "L" else 1
        node = graph[node][index]
        num_steps += 1
    answer_a = num_steps
    starts = [k for k in graph.keys() if k.endswith("A")]
    answer_b = []
    # Solve each path's "Z" problem and calculate the LCM of all
    for i, node in enumerate(starts):
        num_steps = 0
        while not node.endswith("Z"):
            index = 0 if path[num_steps % len(path)] == "L" else 1
            node = graph[node][index]
            num_steps += 1
        answer_b.append(num_steps)
    return answer_a, np.lcm.reduce(answer_b)


assert 6, 6 == wasteland(
    """LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)"""
)

In [ ]:
p.answers = wasteland(p.input_data)